In [12]:
import orjson
import pandas as pd
import numpy as np
from datetime import datetime
import os
from tqdm.auto import tqdm
import csv
import vaex

ERROR:MainThread:vaex:issue loading plot
Traceback (most recent call last):
  File "/home/cdminix/.local/lib/python3.8/site-packages/vaex/__init__.py", line 789, in <module>
    add_namespace = entry.load()
  File "/home/cdminix/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2446, in load
    self.require(*args, **kwargs)
  File "/home/cdminix/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py", line 2469, in require
    items = working_set.resolve(reqs, env, installer, extras=self.extras)
  File "/home/cdminix/anaconda3/lib/python3.8/site-packages/pkg_resources/__init__.py", line 775, in resolve
    raise VersionConflict(dist, req).with_context(dependent_req)
pkg_resources.ContextualVersionConflict: (idna 2.10 (/home/cdminix/anaconda3/lib/python3.8/site-packages), Requirement.parse('idna<2.9,>=2.5'), {'requests'})
ERROR:MainThread:vaex:issue loading astro
Traceback (most recent call last):
  File "/home/cdminix/.local/lib/python3.8/site-packages/va

In [2]:
latlon_df = pd.read_html('https://en.wikipedia.org/wiki/User:Michael_J/County_table', index_col=0)[0]
float_degrees = lambda x: float(x.replace('°','').replace('–','-'))
latlon_df['Latitude'] = latlon_df['Latitude'].apply(float_degrees)
latlon_df['Longitude'] = latlon_df['Longitude'].apply(float_degrees)

In [6]:
def dict_from_line(line):
    values = orjson.loads(line[:-2])
    values = values['features'][0]
    lon, lat = values['geometry']['coordinates'][0], values['geometry']['coordinates'][1]
    fips = latlon_df[
        np.isclose(latlon_df['Longitude'],lon)&
        np.isclose(latlon_df['Latitude'],lat)
    ]['FIPS'].values[0]
    value_dict = {
        'fips': [],
        'date': [],
    }
    for key in values['properties']['parameter']:
        value_dict[key] = []
    key0 = list(values['properties']['parameter'].keys())[0]
    for val in values['properties']['parameter'][key0]:
        date = datetime.strptime(val,'%Y%m%d').date()
        value_dict['fips'].append(fips)
        value_dict['date'].append(date)
        for key in values['properties']['parameter']:
            value_dict[key].append(values['properties']['parameter'][key][val])
    return value_dict

In [4]:
def file_len(fname):
    with open(fname) as f:
        for i, l in tqdm(enumerate(f)):
            pass
    return i + 1

In [5]:
json_len = file_len('data/weather.json')

0it [00:00, ?it/s]

In [10]:
output_path = 'weather_df.csv'
open(output_path, 'w').close()
is_first = True
with open('data/weather.json', 'r') as file:
    with open(output_path, 'w', newline='') as outfile:
        writer = csv.writer(outfile, delimiter = ",")
        for i, line in tqdm(enumerate(file), total=json_len):
            try:
                val_dict = dict_from_line(line)
                if is_first:
                    writer.writerow(val_dict.keys())
                    is_first = False
                writer.writerows(zip(*val_dict.values()))
            except Exception as e:
                print(e)

  0%|          | 0/3111 [00:00<?, ?it/s]

EOF while parsing a value at line 1 column 0: line 1 column 1 (char 0)
EOF while parsing an object at line 1 column 2527088: line 1 column 1 (char 0)
EOF while parsing a value at line 1 column 0: line 1 column 1 (char 0)


In [ ]:
for i, chunk in enumerate(pd.read_csv('weather_df.csv', chunksize=1_000_000)):
    df_chunk = vaex.from_pandas(chunk, copy_index=False)
    export_path = f'data/weather_part_{i}.hdf5'
    df_chunk.export_hdf5(export_path)

In [18]:
df = vaex.open('data/weather_part*')
df['date'] = df['date'].values.astype('datetime64[ns]')
df = df.sort(['fips','date'])
df.export_hdf5('data/weather_df.hdf5')

In [29]:
df.scatter()

RuntimeError: this function is wrapped by a placeholder, you probably want to install vaex-viz